In [8]:
import folium
import pandas as pd
import json

# --- Funções auxiliares ---
def corrigir_link(link):
    if link and not link.startswith("http"):
        return "http://" + link
    return link

def adicionar_descricao(descricao, caracteristica):
    return f"{descricao} (Categoria: {caracteristica})"

# --- Carregamento e processamento dos dados ---
data = pd.read_excel("C:/Users/berna/GitHub/Maps/Brasil/ACERVOS_BR.xlsx")

# Padronizar as categorias da coluna 'Característica do acervo'
data['Característica do acervo'] = data['Característica do acervo'].str.strip().str.lower()

# Corrigir a capitalização das categorias
data['Característica do acervo'] = data['Característica do acervo'].replace({
    'iconográfico': 'Iconográfico',
    'instituição': 'Instituição',
    'hemeroteca': 'Hemeroteca',
    'acervo online': 'Acervo online'
})

# Coordenadas dos estados
coordenadas_estados = {
    'Acre': [-8.77, -70.55], 'Alagoas': [-9.71, -36.73], 'Amazonas': [-3.12, -60.70], 'Bahia': [-12.46, -41.71],
    'Ceará': [-5.20, -39.50], 'Espírito Santo': [-20.30, -40.30], 'Goiás': [-15.60, -49.30], 'Maranhão': [-5.42, -46.30],
    'Mato Grosso': [-12.64, -56.09], 'Mato Grosso do Sul': [-20.51, -54.55], 'Minas Gerais': [-18.10, -44.10],
    'Pará': [-5.60, -52.30], 'Paraíba': [-7.10, -35.50], 'Paraná': [-24.89, -51.55], 'Pernambuco': [-8.28, -35.07],
    'Piauí': [-5.49, -42.28], 'Rio de Janeiro': [-22.91, -43.20], 'Rio Grande do Norte': [-5.39, -36.59],
    'Rio Grande do Sul': [-30.00, -51.22], 'Rondônia': [-10.83, -63.73], 'Roraima': [2.82, -60.67],
    'Santa Catarina': [-27.60, -48.55], 'São Paulo': [-23.55, -46.64], 'Sergipe': [-10.57, -37.45],
    'Tocantins': [-10.17, -48.25]
}

# Dicionários para armazenar os acervos por categoria
acervos_hemeroteca = {}
acervos_iconografico = {}
acervos_acervo_online = {}
acervos_instituicao = {}

# Agrupamento dos acervos por categoria
for index, row in data.iterrows():
    acervo = row['Acervo']
    descricao = row['Descrição']
    estado = row['Estado da Federação']
    site = corrigir_link(row['Página'])
    caracteristica = row['Característica do acervo']
    
    descricao_completa = adicionar_descricao(descricao, caracteristica)
    
    if estado in coordenadas_estados:
        if caracteristica == "Hemeroteca":
            acervos_hemeroteca.setdefault(estado, []).append({
                'nome': acervo,
                'descricao': descricao_completa,
                'site': site
            })
        elif caracteristica == "Iconográfico":
            acervos_iconografico.setdefault(estado, []).append({
                'nome': acervo,
                'descricao': descricao_completa,
                'site': site
            })
        elif caracteristica == "Acervo online":
            acervos_acervo_online.setdefault(estado, []).append({
                'nome': acervo,
                'descricao': descricao_completa,
                'site': site
            })
        elif caracteristica == "Instituição":
            acervos_instituicao.setdefault(estado, []).append({
                'nome': acervo,
                'descricao': descricao_completa,
                'site': site
            })

# Função para gerar o mapa
def gerar_mapa(acervos, categoria, arquivo_saida):
    mapa = folium.Map(location=[-15.78, -47.93], zoom_start=4)
    
    # Adiciona os marcadores no mapa
    for estado, acervos_estado in acervos.items():
        latitude, longitude = coordenadas_estados[estado]
        popup_content = f"<a href='#' onclick=\"loadAcervos('{estado}'); return false;\">Ver acervos de {estado}</a>"
        popup = folium.Popup(popup_content, max_width=300)
        
        folium.Marker(
            location=[latitude, longitude],
            popup=popup,
            tooltip=f"{estado} ({len(acervos_estado)} acervo(s))"
        ).add_to(mapa)

    # Criação do HTML e JavaScript para interatividade
    acervos_json = json.dumps(acervos)  # Dados dos acervos em formato JSON

    custom_html = f"""
    <style>
      /* Estilo para os painéis */
      #acervo-list {{
          position: absolute;
          top: 10px;
          left: 10px;
          width: 350px;  /* Aumenta a largura da janela */
          height: 400px;
          overflow-y: auto;
          background-color: white;
          z-index: 1000;
          padding: 5px;
          border: 2px solid black;
          font-size: 20px;
      }}
      #acervo-detail {{
          position: absolute;
          top: 10px;
          right: 10px;
          width: 50%;  /* Define 50% da largura do mapa */
          height: 80%;  /* Aumenta a altura da janela */
          overflow-y: auto;
          background-color: white;
          z-index: 2000;
          padding: 10px;
          border: 4px solid black;
          font-size: 20px;
      }}
      #acervo-detail iframe {{
          width: 100%;  /* Garante que o iframe ocupe toda a largura */
          height: 600px;  /* Ajusta a altura do iframe para que a cabeça do site seja visível */
          border: none;
      }}
      .destaque {{
          font-size: 26px;
          font-weight: bold;
          color: #2d2d2d;
          text-align: center;
          margin-bottom: 20px;
      }}
    </style>

    <!-- Painel da lista de acervos -->
    <div id="acervo-list">
      <div class="destaque">Olá Usuário! Clique em qualquer região no mapa para carregar a lista de instituições!</div>
      <p><span style="font-size: 18px;">Última atualização: FEV.2025.</span></p>
    </div>

    <!-- Painel de detalhes do acervo -->
    <div id="acervo-detail">
      <div class="destaque">Ao passar o mouse sobre o nome da instituição, os detalhes serão exibidos aqui! Mas atenção: alguns sites possuem proteções de segurança.</div>
      <p><span style="font-size: 18px;">Caso o site não seja exibido, clique em VISITAR SITE.</span></p>
      <p><span style="font-size: 18px;">Última atualização: FEV.2025.</span></p>
    </div>

    <script>
    // Dados dos acervos, injetados a partir do Python
    var acervos_data = {acervos_json};

    // Função para carregar os acervos de um estado no painel da lista
    function loadAcervos(state) {{
        var listDiv = document.getElementById("acervo-list");
        listDiv.innerHTML = "<h4>Acervos de " + state + "</h4>";
        if(acervos_data[state]) {{
            acervos_data[state].forEach(function(acervo) {{
                var a = document.createElement("a");
                a.href = "#";
                a.textContent = acervo.nome;
                a.style.display = "block";
                a.style.marginBottom = "5px";
                a.onclick = function() {{
                    showAcervoDetail(acervo);
                }};
                listDiv.appendChild(a);
            }});
        }} else {{
            listDiv.innerHTML += "<p>Nenhum acervo encontrado.</p>";
        }}
    }}

    // Função para exibir os detalhes do acervo no painel de detalhes
    function showAcervoDetail(acervo) {{
        var detailDiv = document.getElementById("acervo-detail");
        detailDiv.innerHTML = "<h4>" + acervo.nome + "</h4>" +
                              "<p><b>Descrição:</b> " + acervo.descricao + "</p>" +
                              "<p><a href='" + acervo.site + "' target='_blank'>Visitar site</a></p>" +
                              "<iframe src='" + acervo.site + "'></iframe>";  // Atualiza o iframe
    }}
    </script>
    """

    # Adiciona o HTML customizado ao mapa
    mapa.get_root().html.add_child(folium.Element(custom_html))

    # Salva o mapa em um arquivo HTML
    mapa.save(arquivo_saida)
    print(f"Mapa '{categoria}' gerado com sucesso!")

# Gerar mapas para cada categoria
gerar_mapa(acervos_hemeroteca, 'Hemeroteca', 'Hemeroteca_mapa.html')
gerar_mapa(acervos_iconografico, 'Iconográfico', 'Iconografico_mapa.html')
gerar_mapa(acervos_acervo_online, 'Acervo_online', 'AcervoOnline_mapa.html')
gerar_mapa(acervos_instituicao, 'Instituição', 'Instituicao_mapa.html')


Mapa 'Hemeroteca' gerado com sucesso!
Mapa 'Iconográfico' gerado com sucesso!
Mapa 'Acervo_online' gerado com sucesso!
Mapa 'Instituição' gerado com sucesso!
